# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [28]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet','stopwords', 'averaged_perceptron_tagger'])


import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.metrics import f1_score

[nltk_data] Downloading package punkt to C:\Users\Telu
[nltk_data]     Teruno\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Telu
[nltk_data]     Teruno\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Telu
[nltk_data]     Teruno\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Telu Teruno\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [29]:
# load data from database
def load_data():
    from sqlalchemy import create_engine
    engine = create_engine('sqlite:///InsertDatabaseName.db')
    df = pd.read_sql_table('Messages', 'sqlite:///NaturalDisastersMsgs.db') 
    df.head()
    X = df['message']
    Y = df.drop(['id', 'message','original', 'genre'], axis=1)
    return X, Y

### 2. Write a tokenization function to process your text data

In [30]:
def tokenize(text):
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    tokens = tokenizer.tokenize(text)
    tokens = [t for t in tokens if t not in stopwords.words("english")]
    lemmatizer = WordNetLemmatizer()
        
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tok = PorterStemmer().stem(tok)
        clean_tokens.append(clean_tok)

    return clean_tokens

In [31]:
#test
X, y = load_data()
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti'] 

Is the Hurricane over or is it not over
['Is', 'hurrican'] 

Looking for someone but no name
['look', 'someon', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['UN', 'report', 'leogan', '80', '90', 'destroy', 'onli', 'hospit', 'St', 'croix', 'function', 'need', 'suppli', 'desper'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'countri', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [32]:
def display_results():
    f1_scores = []
    for i in range(len(y_test.columns)):
        print(y_test.columns[i])
        f1 = f1_score(y_test.values[:,i], y_pred[:,i], average='weighted')
        f1_scores.append(f1)
        print(classification_report(y_test.values[:,i], y_pred[:,i]))
    print("The average f1 weighted score of all the columns is", np.mean(f1_scores))
    

In [33]:
X, y = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y)

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

display_results()

related
              precision    recall  f1-score   support

           0       0.57      0.32      0.41      1525
           1       0.81      0.92      0.86      4973
           2       0.31      0.34      0.32        56

    accuracy                           0.77      6554
   macro avg       0.56      0.53      0.53      6554
weighted avg       0.75      0.77      0.75      6554

request
              precision    recall  f1-score   support

           0       0.84      0.99      0.91      5443
           1       0.76      0.10      0.18      1111

    accuracy                           0.84      6554
   macro avg       0.80      0.55      0.55      6554
weighted avg       0.83      0.84      0.79      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6515
           1       0.00      0.00      0.00        39

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      655

C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6137
           1       0.78      0.05      0.09       417

    accuracy                           0.94      6554
   macro avg       0.86      0.52      0.53      6554
weighted avg       0.93      0.94      0.91      6554

food
              precision    recall  f1-score   support

           0       0.89      1.00      0.94      5809
           1       0.74      0.08      0.14       745

    accuracy                           0.89      6554
   macro avg       0.82      0.54      0.54      6554
weighted avg       0.88      0.89      0.85      6554

shelter
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      6017
           1       0.76      0.04      0.08       537

    accuracy                           0.92      6554
   macro avg       0.84      0.52      0.52      6554
weighted avg       0.91      0.92      0.89      6554

clothi

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [27]:
display_results()

0
              precision    recall  f1-score   support

           0       0.48      0.58      0.52      1532
           1       0.85      0.81      0.83      4969
           2       0.71      0.19      0.30        53

    accuracy                           0.75      6554
   macro avg       0.68      0.52      0.55      6554
weighted avg       0.77      0.75      0.76      6554

1
              precision    recall  f1-score   support

           0       0.89      0.96      0.93      5466
           1       0.69      0.42      0.52      1088

    accuracy                           0.87      6554
   macro avg       0.79      0.69      0.72      6554
weighted avg       0.86      0.87      0.86      6554

2
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6530
           1       0.00      0.00      0.00        24

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg  


30
              precision    recall  f1-score   support

           0       0.92      0.99      0.95      5914
           1       0.69      0.19      0.30       640

    accuracy                           0.91      6554
   macro avg       0.80      0.59      0.63      6554
weighted avg       0.90      0.91      0.89      6554

31
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6494
           1       0.75      0.05      0.09        60

    accuracy                           0.99      6554
   macro avg       0.87      0.52      0.54      6554
weighted avg       0.99      0.99      0.99      6554

32
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      5951
           1       0.80      0.47      0.59       603

    accuracy                           0.94      6554
   macro avg       0.88      0.73      0.78      6554
weighted avg       0.94      0.94      0.93      6554

33
      

### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
parameters = {
    'tfidf__norm': ['l1', 'l2']}
#     'tfidf__use_idf': ['True', 'False']
#     'clf__estimator': ['KNeighborsClassifier()', 'RandomForestClassifier()']}

cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, y_train)

y_pred = cv.predict(X_test)

C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [8]:
for i in range(len(y_test.columns)):
    print(y_test.columns[i])
    print(classification_report(y_test.values[:,i], y_pred[:,i]))
    
#It seems there´s no change from the first attempt so the default parameter of tfidf__norm, that is l2 is the best
#I can´t make more tests because just this took like one hour running.


0
              precision    recall  f1-score   support

           0       0.74      0.16      0.26      1544
           1       0.81      0.10      0.18      4961
           2       0.01      0.94      0.02        49

    accuracy                           0.12      6554
   macro avg       0.52      0.40      0.15      6554
weighted avg       0.79      0.12      0.20      6554

1
              precision    recall  f1-score   support

           0       0.84      0.99      0.91      5452
           1       0.71      0.09      0.16      1102

    accuracy                           0.84      6554
   macro avg       0.78      0.54      0.54      6554
weighted avg       0.82      0.84      0.79      6554

2
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1       0.00      0.00      0.00        30

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg  

C:\Users\Telu Teruno\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



12
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      5975
           1       0.72      0.04      0.08       579

    accuracy                           0.91      6554
   macro avg       0.82      0.52      0.52      6554
weighted avg       0.90      0.91      0.88      6554

13
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6447
           1       0.25      0.01      0.02       107

    accuracy                           0.98      6554
   macro avg       0.62      0.50      0.50      6554
weighted avg       0.97      0.98      0.98      6554

14
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6425
           1       1.00      0.02      0.03       129

    accuracy                           0.98      6554
   macro avg       0.99      0.51      0.51      6554
weighted avg       0.98      0.98      0.97      6554

15
      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
#I´m going to change the machine learning algorithm and see how it works
X, y = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y)

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(tree.DecisionTreeClassifier()))
])

# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

In [12]:
for i in range(len(y_test.columns)):
    print(y_test.columns[i])
    print(classification_report(y_test.values[:,i], y_pred[:,i]))
    
#In general terms the decision trees algorithm gets better results than the KNeighborsClassifier so we keep this last model as our best


0
              precision    recall  f1-score   support

           0       0.51      0.50      0.50      1516
           1       0.86      0.85      0.85      4991
           2       0.18      0.51      0.26        47

    accuracy                           0.76      6554
   macro avg       0.51      0.62      0.54      6554
weighted avg       0.77      0.76      0.77      6554

1
              precision    recall  f1-score   support

           0       0.91      0.92      0.92      5426
           1       0.60      0.56      0.58      1128

    accuracy                           0.86      6554
   macro avg       0.75      0.74      0.75      6554
weighted avg       0.86      0.86      0.86      6554

2
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1       0.07      0.07      0.07        30

    accuracy                           0.99      6554
   macro avg       0.53      0.53      0.53      6554
weighted avg  


25
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.07      0.06      0.07        32

    accuracy                           0.99      6554
   macro avg       0.53      0.53      0.53      6554
weighted avg       0.99      0.99      0.99      6554

26
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      6480
           1       0.16      0.12      0.14        74

    accuracy                           0.98      6554
   macro avg       0.57      0.56      0.56      6554
weighted avg       0.98      0.98      0.98      6554

27
              precision    recall  f1-score   support

           0       0.96      0.97      0.96      6258
           1       0.14      0.11      0.13       296

    accuracy                           0.93      6554
   macro avg       0.55      0.54      0.55      6554
weighted avg       0.92      0.93      0.93      6554

28
      

### 9. Export your model as a pickle file

In [15]:
import pickle
pickle.dump(pipeline, open('finalized_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.